## Setup

In [ ]:
import sys
from pathlib import Path

from aocd import get_data, submit

In [2]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [3]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=6)

## Part a

In [ ]:
# Imports
from enum import Enum
from time import sleep

from common.utils.dict_grid import print_grid, text_to_dict
from IPython.display import clear_output


In [5]:
# Classes
class GridChar(str, Enum):
    """Enum with grid characters."""

    # Characters from the problem statement
    empty = "."
    wall = "#"
    guard = "^"

    # Custom characters
    visited = "X"
    guard_emoji = "💂"


class InfiniteTraversalError(RuntimeError):
    """Custom error for infinite loops in the grid traversal."""

In [7]:
def traverse_grid(
    grid: dict[complex, str],
    direction: complex = 1j,
    *,
    show_grid: bool = False,
    step_count: int = 1_000,
    max_steps: int = 1_000_000,
) -> list[complex]:
    """Traverse the grid and return the list of visited cells.

    Args:
        grid (dict[complex, str]): The grid to traverse.
        direction (complex, optional): The starting direction of the guard in complex notation. Defaults to 1j (up).
        show_grid (bool, optional): Whether to print the grid at each step count. Defaults to False.
        step_count (int, optional): The number of steps between grid prints. Defaults to 1_000.
        max_steps (int, optional): The maximum number of steps before raising an error. Defaults to 1_000_000.

    Raises:
        InfiniteTraversalError: Raised when the traversal exceeds the maximum number of steps.

    Returns:
        list[complex]: The list of visited cells.

    """
    # Copy the grid to avoid modifying the original
    grid_copy = grid.copy()

    # Get the starting position of the guard
    position = next(k for k, v in grid.items() if v == GridChar.guard)

    # Initialize the list of traversed cell positions
    traversed_cells: list[complex] = []

    # Loop until the guard exits the grid
    for i in range(max_steps):
        if show_grid and i % step_count == 0:
            # Print the current position and the grid
            clear_output(wait=True)
            print(f"Step {i}, position: {position.real:.0f}, {position.imag:.0f}")  # Print is used to show the grid
            print_grid(grid_copy)
            sleep(0.05)

        # Get the value of the next position
        next_position: str | None = grid_copy.get(position + direction)

        if next_position is None:
            # Guard will exit the grid. Add the last cell to the list and end the loop.
            return [*traversed_cells, position]

        if next_position == GridChar.wall:
            # Turn right and check the next step
            direction *= -1j
            continue

        # Mark the cell as visited
        traversed_cells.append(position)
        grid_copy[position] = GridChar.visited

        # Take step forward and update the grid
        position += direction
        grid_copy[position] = GridChar.guard_emoji

    # Guard against large loops
    err_msg: str = f"Grid traversal longer than {max_steps:,} steps, suspected infinite loop."
    raise InfiniteTraversalError(err_msg)

In [ ]:
# Parse input into a dictionary with complex number keys to serve as a grid.
grid = text_to_dict(data)

# Traverse the grid and show the grid while traversing
traversed_cells = traverse_grid(grid, show_grid=True, step_count=100)

# Print the number of unique cells visited
route_length = len(set(traversed_cells))

In [ ]:
# Submit answer
submit(route_length, part="a", day=6, year=2024)

## Part b

In [23]:
# Imports
from tqdm import tqdm

In [ ]:
# Find the positions of the open cells
open_cells = [k for k, v in grid.items() if v == GridChar.empty]

obstruction_cells: list[complex] = []

for position in tqdm(open_cells):
    try:
        traverse_grid({**grid, position: GridChar.wall}, max_steps=50_000)
    except InfiniteTraversalError:
        obstruction_cells.append(position)

In [ ]:
# Submit answer
submit(len(obstruction_cells), part="b", day=6, year=2024)